## Qubits

In [3]:
import cirq

qubit = cirq.NamedQubit("myqubit")

#create a Hadamard gate
circuit = cirq.Circuit(cirq.H(qubit))
print(circuit)

result = cirq.Simulator().simulate(circuit)

print(result)

myqubit: ───H───
measurements: (no measurements)
output vector: 0.707|0⟩ + 0.707|1⟩


In [9]:
test_qubit = cirq.LineQubit.range(3)
test_qubit

[cirq.LineQubit(0), cirq.LineQubit(1), cirq.LineQubit(2)]

## Gates and Operations

A **gate** is an effect that can be applied to a collection of qubits (which are objects with a **Qid**). 

Gates can be applied to qubits by calling their **on** method or by calling the gate on the qubits. 

The object created by these calls is an **Operation**.

In [10]:
a,b,c = cirq.LineQubit.range(3)

In [13]:
print(cirq.H(b))

H(1)


In [15]:
print(cirq.CNOT(b,c))

CNOT(1, 2)


In [16]:
print(cirq.CNOT(a,b))

CNOT(0, 1)


In [18]:
print(cirq.H(a))

H(0)


In [20]:
print(cirq.measure(a,b))

cirq.MeasurementGate(2, '0,1', ())(0, 1)


## Circuits

Sometimes qubits are arranged on the vertices of a square grid, like Google's Xmon devices. 

Here's a 3X3 qubit grid arrangement:

In [37]:
qubits = [ cirq.GridQubit(x,y) for x in range(3) for y in range(3) ]

qubits

[cirq.GridQubit(0, 0),
 cirq.GridQubit(0, 1),
 cirq.GridQubit(0, 2),
 cirq.GridQubit(1, 0),
 cirq.GridQubit(1, 1),
 cirq.GridQubit(1, 2),
 cirq.GridQubit(2, 0),
 cirq.GridQubit(2, 1),
 cirq.GridQubit(2, 2)]

In [38]:
#let's instantiate a Pauli X gate and apply it to a single qubit

x_gate = cirq.X
x_op = x_gate(qubits[0])

print(x_op)

X((0, 0))


A **Moment** is a collection of operations on different qubits, representing a time slice.

In [39]:
#a moment with a Pauli X gate and a CZ gate operating on three qubits

#instantiate
cz = cirq.CZ(qubits[0],qubits[1])
x = cirq.X(qubits[2])
moment = cirq.Moment([x,cz])

print(moment)

  ╷ 0 1 2
╶─┼───────
0 │ @─@ X
  │


In [43]:
#a more complicated circuit composed of two moments

#instantiate the gates
cz01 = cirq.CZ(qubits[0],qubits[1])
x2 = cirq.X(qubits[2])
cz12 = cirq.CZ(qubits[1],qubits[2])

#instantiate the moments
moment0 = cirq.Moment([cz01,x2])
moment1 = cirq.Moment([cz12])

#create the circuit
circuit = cirq.Circuit((moment0,moment1))

print(circuit)

(0, 0): ───@───────
           │
(0, 1): ───@───@───
               │
(0, 2): ───X───@───


So a circuit is a series of moments. But moments can be tedious to code by hand, so Cirq provides a few differents shorthand ways to create a circuit.

Here we can use the append method:

In [53]:
from cirq.ops import CZ, H

q0, q1, q2 = [ cirq.GridQubit(i,0) for i in range(3) ]

circuit = cirq.Circuit()
circuit.append([CZ(q0,q1), H(q2)])

print(circuit)

(0, 0): ───@───
           │
(1, 0): ───@───

(2, 0): ───H───


In [54]:
#we can append again

circuit.append([H(q0), CZ(q1,q2)])

print(circuit)

(0, 0): ───@───H───
           │
(1, 0): ───@───@───
               │
(2, 0): ───H───@───


In [56]:
#or we can just do all the above at once

circuit = cirq.Circuit()
circuit.append([CZ(q0,q1), H(q2), H(q0), CZ(q1,q2)])

print(circuit)

(0, 0): ───@───H───
           │
(1, 0): ───@───@───
               │
(2, 0): ───H───@───


*Circuit.append* and its cousin *Circuit.insert* both take an argument called **InsertStrategy**. This defines for operations are placed in a circuit when requested to be inserted at a given location. There are 4 strategies:

* InsertStrategy.EARLIEST
* InsertStrategy.NEW
* InsertStrategy.INLINE
* InsertStrategy.NEW_THEN_INLINE

In [57]:
from cirq.circuits import InsertStrategy

In [69]:
circuit = cirq.Circuit()
circuit.append([CZ(q0,q1)])
circuit.append([H(q0),H(q2)], strategy = InsertStrategy.EARLIEST)

print(f"EARLIEST: \n{circuit}")

circuit = cirq.Circuit()
circuit.append([CZ(q0,q1)])
circuit.append([H(q0),H(q2)], strategy = InsertStrategy.NEW)

print(f"\nNEW: \n{circuit}")

circuit = cirq.Circuit()
circuit.append([CZ(q0,q1)])
circuit.append([H(q0),H(q2)], strategy = InsertStrategy.INLINE)

print(f"\nINLINE: \n{circuit}")

circuit = cirq.Circuit()
circuit.append([CZ(q0,q1)])
circuit.append([H(q0),H(q2)], strategy = InsertStrategy.NEW_THEN_INLINE)

print(f"\nNEW_THEN_INLINE: \n{circuit}")

EARLIEST: 
(0, 0): ───@───H───
           │
(1, 0): ───@───────

(2, 0): ───H───────

NEW: 
(0, 0): ───@───H───────
           │
(1, 0): ───@───────────

(2, 0): ───────────H───

INLINE: 
(0, 0): ───@───H───
           │
(1, 0): ───@───────

(2, 0): ───────H───

NEW_THEN_INLINE: 
(0, 0): ───@───H───
           │
(1, 0): ───@───────

(2, 0): ───────H───


In [75]:
#another example of NEW
circuit = cirq.Circuit()
circuit.append([H(q0),H(q1),H(q2)])

print(f"Original:\n{circuit}")

circuit = cirq.Circuit()
circuit.append([H(q0),H(q1),H(q2)], strategy=InsertStrategy.NEW)

print(f"\nNEW:\n{circuit}")

Original:
(0, 0): ───H───

(1, 0): ───H───

(2, 0): ───H───

NEW:
(0, 0): ───H───────────

(1, 0): ───────H───────

(2, 0): ───────────H───


In [83]:
#another example of INLINE

circuit = cirq.Circuit()
circuit.append([CZ(q1,q2)])
circuit.append([CZ(q1,q2)])
circuit.append([H(q0),H(q1),H(q2)])

print(f'Original:\n{circuit}')

circuit = cirq.Circuit()
circuit.append([CZ(q1,q2)])
circuit.append([CZ(q1,q2)])
circuit.append([H(q0),H(q1),H(q2)], strategy=InsertStrategy.INLINE)

print(f'\nINLINE:\n{circuit}')

Original:
(0, 0): ───H───────────

(1, 0): ───@───@───H───
           │   │
(2, 0): ───@───@───H───

INLINE:
(0, 0): ───────H───────

(1, 0): ───@───@───H───
           │   │
(2, 0): ───@───@───H───


Note that the first H-gate was inserted into the previous moment (2nd moment), but the H-gates on the 2nd and 3rd qubits cannot be inserted so a new moment is created

In [87]:
#another example of NEW_THEN_INLINE

circuit = cirq.Circuit()
circuit.append([H(q0)])
circuit.append([CZ(q1,q2), H(q0)])

print(f"Original:\n{circuit}")

circuit = cirq.Circuit()
circuit.append([H(q0)])
circuit.append([CZ(q1,q2), H(q0)], strategy = InsertStrategy.NEW_THEN_INLINE)

print(f"\nNEW_THEN_INLINE:\n{circuit}")

Original:
(0, 0): ───H───H───

(1, 0): ───@───────
           │
(2, 0): ───@───────

NEW_THEN_INLINE:
(0, 0): ───H───H───

(1, 0): ───────@───
               │
(2, 0): ───────@───


In [92]:
#the circuit.append() method can take more than just list values:

def my_layer():
    yield CZ(q0,q1)
    yield [H(q) for q in (q0,q1,q2)]
    yield [CZ(q1,q2)]
    yield [H(q0), [CZ(q1,q2)]]
    
circuit = cirq.Circuit()
circuit.append(my_layer())

print(f"The circuit:\n{circuit}")
print("\nThe details:")

for x in my_layer():
    print(x)

The circuit:
(0, 0): ───@───H───H───────
           │
(1, 0): ───@───H───@───@───
                   │   │
(2, 0): ───H───────@───@───

The details:
CZ((0, 0), (1, 0))
[cirq.H(cirq.GridQubit(0, 0)), cirq.H(cirq.GridQubit(1, 0)), cirq.H(cirq.GridQubit(2, 0))]
[cirq.CZ(cirq.GridQubit(1, 0), cirq.GridQubit(2, 0))]
[cirq.H(cirq.GridQubit(0, 0)), [cirq.CZ(cirq.GridQubit(1, 0), cirq.GridQubit(2, 0))]]


The above uses the concept of an **OPT_TREE** which is not a class but a *contract*

The idea of an OPT_TREE is, the input is iteratively flattened into a list of operations. Here's an example:

In [94]:
circuit = cirq.Circuit(H(q0),H(q1))

circuit

(0, 0): ───H───

(1, 0): ───H───

Circuits can be iterated over and sliced. Each item in the iteration is a moment

In [96]:
circuit = cirq.Circuit(H(q0), CZ(q0,q1))

for moment in circuit:
    print(moment)
    
print(circuit)

  ╷ 0
╶─┼───
0 │ H
  │
  ╷ 0
╶─┼───
0 │ @
  │ │
1 │ @
  │
(0, 0): ───H───@───
               │
(1, 0): ───────@───


In [99]:
circuit = cirq.Circuit(H(q0), CZ(q0,q1), H(q1), CZ(q0,q1))

print(f"Original:\n{circuit}")

print(f"\nSliced:\n{circuit[1:3]}")

Original:
(0, 0): ───H───@───────@───
               │       │
(1, 0): ───────@───H───@───

Sliced:
(0, 0): ───@───────
           │
(1, 0): ───@───H───


## Simulation

In [101]:
#let's create a circuit

#first the qubits
q0 = cirq.GridQubit(0,0)
q1 = cirq.GridQubit(1,0)

#create a generator function
def basic_circuit(meas=True):
    sqrt_x = cirq.X**0.5
    yield sqrt_x(q0), sqrt_x(q1)
    yield cirq.CZ(q0,q1)
    yield sqrt_x(q0), sqrt_x(q1)
    if meas:
        yield cirq.measure(q0, key='q0'), cirq.measure(q1, key='q1')
        
circuit = cirq.Circuit()
circuit.append(basic_circuit())

print(circuit)

(0, 0): ───X^0.5───@───X^0.5───M('q0')───
                   │
(1, 0): ───X^0.5───@───X^0.5───M('q1')───


In [103]:
#create a Simulator
from cirq import Simulator

In [106]:
#instantiate Simulator and run it
simulator = Simulator()
result = simulator.run(circuit)

#the .run() method returns a result
print(result)

q0=1
q1=1


In [107]:
#another run may (or not) get different results
result = simulator.run(circuit)

print(result)

q0=1
q1=0


In [108]:
#here's what the statevector looks like
import numpy as np

In [110]:
circuit = cirq.Circuit()
circuit.append(basic_circuit(False))

result = simulator.simulate(circuit, qubit_order=[q0,q1])

print(result.final_state_vector)

[0.5+0.j  0. +0.5j 0. +0.5j 0.5+0.j ]


##### Calculating expectation values(!!)

In [113]:
XX_obs = cirq.X(q0) * cirq.X(q1)
ZZ_obs = cirq.Z(q0) * cirq.Z(q1)

ev_list = simulator.simulate_expectation_values(cirq.Circuit(basic_circuit(False)),
                                               observables = [XX_obs, ZZ_obs])

print(ev_list)

[(1+0j), 0j]


##### Qubit and amplitude ordering

The mapping from the order of qubits to the order of amplitudes in the wave function is basically the same order as the Kronecker delta function

In [115]:
outside = [1,10]
inside = [1,2]

print(np.kron(outside,inside))

[ 1  2 10 20]


In [116]:
i = 0

for first in [0,1]:
    for second in [0,1]:
        print(f"amps[{i}] is for first = {first}, second = {second}")
        i += 1

amps[0] is for first = 0, second = 0
amps[1] is for first = 0, second = 1
amps[2] is for first = 1, second = 0
amps[3] is for first = 1, second = 1


In [121]:
#we can check this is the right odering with a circuit that flips one of the qubits

q_stay = cirq.NamedQubit('q_stay')
q_flip = cirq.NamedQubit('q_flip')
my_circuit = cirq.Circuit(cirq.X(q_flip))

#first qubit flipped
result = simulator.simulate(my_circuit, qubit_order = [q_flip, q_stay])
print(abs(result.final_state_vector).round(3))

#second qubit flipped
result = simulator.simulate(my_circuit, qubit_order = [q_stay, q_flip])
print(abs(result.final_state_vector).round(3))

[0. 0. 1. 0.]
[0. 1. 0. 0.]


In [135]:
#you can inspect or even modify the state at different steps in the circuit

circuit = cirq.Circuit()
circuit.append(basic_circuit())
for i, step in enumerate(simulator.simulate_moment_steps(circuit)):
    print('state at step %d: %s' % (i, np.around(step.state_vector(), 3)))

state at step 0: [0. +0.5j 0.5+0.j  0.5+0.j  0. -0.5j]
state at step 1: [0. +0.5j 0.5+0.j  0.5+0.j  0. +0.5j]
state at step 2: [0.5+0.j  0. +0.5j 0. +0.5j 0.5+0.j ]
state at step 3: [0.+0.j 0.+0.j 0.+0.j 1.+0.j]


In [140]:
#you can also split the circuit into chunks
#and pass the results of each chunk as the initial state for each new chunk

chunks = [ cirq.Circuit(moment) for moment in basic_circuit() ]
next_state = 0 #representing the all-zero state

for i, chunk in enumerate(chunks):
    result = simulator.simulate(chunk, initial_state = next_state)
    next_state = result.final_state_vector
    print(f"state at step {i}: {np.around(next_state,3)}")

state at step 0: [0. +0.5j 0.5+0.j  0.5+0.j  0. -0.5j]
state at step 1: [0. +0.5j 0.5+0.j  0.5+0.j  0. +0.5j]
state at step 2: [0.5+0.j  0. +0.5j 0. +0.5j 0.5+0.j ]
state at step 3: [0.+0.j 0.+0.j 0.+0.j 1.+0.j]


##### Parametrized values and studies

Cirq supports gates which can have a **Symbol** value, values that can be resolved at runtime

For simulators, these values are resolved by a *cirq.ParamResolver()* method

In [141]:
import sympy

In [145]:
#creating a gate
rot_w_gate = cirq.X ** sympy.Symbol('x')

circuit = cirq.Circuit()
circuit.append([rot_w_gate(q0), rot_w_gate(q1)])

for y in range(5):
    resolver = cirq.ParamResolver({'x': y / 4.0})
    result = simulator.simulate(circuit, resolver)
    print(np.round(result.final_state_vector,2))

[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
[ 0.6 +0.6j   0.25-0.25j  0.25-0.25j -0.1 -0.1j ]
[0. +0.5j 0.5+0.j  0.5+0.j  0. -0.5j]
[-0.1 +0.1j   0.25+0.25j  0.25+0.25j  0.6 -0.6j ]
[0.+0.j 0.+0.j 0.+0.j 1.+0.j]


In [151]:
#paramterized values are most useful in defining a sweep
#and a sweep is a sequence of trials

resolvers = [ cirq.ParamResolver({'x': y / 2.0}) for y in range(3) ]

circuit = cirq.Circuit()
circuit.append([rot_w_gate(q0), rot_w_gate(q1)])
circuit.append([cirq.measure(q0, key = 'q0'), cirq.measure(q1, key = 'q1')])

results = simulator.run_sweep(program = circuit, params = resolvers, repetitions = 2)

i=1
for result in results:
    print(f'Trial {i}:\n{result}\n')
    i+=1

Trial 1:
q0=00
q1=00

Trial 2:
q0=10
q1=11

Trial 3:
q0=11
q1=11



##### Mixed state simulations

This simulator puts a state in equal superposition of 0 and 1, then applies amplitude damping which takes 1 to 0 with a specified probability (0.2 here)

So about 20% of the 1s are converted to 0s, leading to a ~60% probability of a 0 measurement

In [159]:
q = cirq.NamedQubit('a')

circuit = cirq.Circuit(cirq.H(q), cirq.amplitude_damp(0.2)(q), cirq.measure(q))

simulator = cirq.DensityMatrixSimulator()
result = simulator.run(circuit,repetitions=100)

print(f'Result is:\n{result}\n')
print(f"Counter:\n{result.histogram(key='a')}")

Result is:
a=1001111011000011101101000100000010001000110011010001100100010000000010101011001110000010111010111100

Counter:
Counter({0: 58, 1: 42})


In [162]:
#here's a density matrix

q = cirq.NamedQubit('a')

circuit = cirq.Circuit(cirq.H(q), cirq.amplitude_damp(0.2)(q))

simulator = cirq.DensityMatrixSimulator()
result = simulator.simulate(circuit)

print(np.around(result.final_density_matrix,3))

[[0.6  +0.j 0.447+0.j]
 [0.447+0.j 0.4  +0.j]]
